1D spline function of degree $k$ is defined by the knot vector $t$ and coefficients, $c$, via

$$
S(x) = \sum_{j=1}^{n} c_j B_{j; t}(x)
$$

where $B_j; t(x)$ are the b-spline basis elements. A key feature of the b-spline basis elements is that they are _localized_ : $B_{j; t}(x) \neq 0$ only for $t_{j-k} \leqslant x \leqslant t_{j+1}$. Therefore, the sum above has only $k+1$ non-zero terms for a given value of $x$.

For an N-dimensional generalization, we define $N$ knots vectors, $t_d$, for $d=1,\dots, N$. To lighten the notation where possible, we from now on use lower subscripts, $t_d$, to enumerate knot _vectors_, not their individual elements. To index the latter we will use the double index: $t_{d; j}$ is the $j$-th element of $t_d$.

Each knot vector defines its set of $B_{j_d; t_d}$ basis elements, and the tensor product spline function is a linear combination of their tensor products. Specifically, for an $N$-dimensional vector $\vec{x} = (x_1, \cdots, x_N)$, a tensor product spline function is defined via

$$
S_N(\vec{x}) = \sum_{j_1}^{n_1}\cdots \sum_{j_N=1}^{n_N} c_{j_1, \cdots, j_N} B_{j_1; t_1}(x_1) \cdots B_{j_N; t_N}(x_N) \qquad (1)
$$

Note the coefficients, $c_{j_1, j_2, \cdots, j_N}$,  now carry $N$ indices. Also for simplicity we assumed that the spline degrees is $k$ for all dimensions (this can be easily generalized). 

For a given value of input vector, $\vec{x}$, the sum above has at most $(k+1)^N$ non-zero elements. Let $s_d$ are indices into $t$ vectors, such that

$$
t_{d; s_d} \leqslant x_d \leqslant t_{d; s_d + 1}    \qquad (2)
$$

Then, the $S_N(\vec{x})$ function is identically

$$
S_N(\vec{x}) = \sum_{i_1 = s_1 - k +1}^{s_1+1} \cdots \sum_{i_N = s_N - k +1}^{s_N + 1} %
         c_{i_1, \cdots, i_N} B_{i_1; t_1}(x_1) \cdots B_{i_N; t_N}(x_N)  \qquad (3)
$$

Thus, computing the value of $S_N(\vec{x})$ is a two-step process: first, find the indices, $s_d$; second, form linear combinations (3). 

For performance reasons, the second step is best done on a flattened array of coefficients. The correspondence between the "flat" index and indices $i_1, \cdots, i_N$ is given by `np.unravel_index` and `np.ravel_multi_index`. Moreover, since the shape of the $c$ array is fixed, these functions can be inlined into stride manipulations.

# TODO

1. NdBSpline: a tuple of non-zero b-spline vectors
2. *[DONE]* Tests with 2D
3. *[DONE]* and 3D
4. *[DONE]* vector of `x`   + tests
5. *[DONE]* trailing dimensions of `c`: reshape, loop, reshape back + tests
6. *[DONE]* test 2D random, non-separable
7. *[DONE]* test mixed degrees: k = [2, 3] etc
8. *[DONE]* test against NdBSpline0
9. *[DONE]* consistency checks of input data
10. complex-valued coefficients
--

11. *[DONE]* get rid of `itertools.product` : iterate over c.flat (subarray!)
12. *[DONE]* change np.unravel_index / ravel_multi_index to stride manipulation (indices / pointers!)

In [1]:
import operator
import itertools

import numpy as np

In [2]:
def B(x, k, i, t):
    if k == 0:
        return 1.0 if t[i] <= x < t[i+1] else 0.0
    if t[i+k] == t[i]:
        c1 = 0.0
    else:
        c1 = (x - t[i])/(t[i+k] - t[i]) * B(x, k-1, i, t)
    if t[i+k+1] == t[i+1]:
        c2 = 0.0
    else:
        c2 = (t[i+k+1] - x)/(t[i+k+1] - t[i+1]) * B(x, k-1, i+1, t)
    return c1 + c2

def bspline(x, t, c, k):
    n = len(t) - k - 1
    assert (n >= k+1) and (len(c) >= n)
    return sum(c[i] * B(x, k, i, t) for i in range(n))

In [3]:
def _naive_eval(x, t, c, k):
    """
    Naive B-spline evaluation. Useful only for testing!
    """
    if x == t[k]:
        i = k
    else:
        i = np.searchsorted(t, x) - 1
    assert t[i] <= x <= t[i+1]
    assert i >= k and i < len(t) - k
    return sum(c[i-j] * B(x, k, i-j, t) for j in range(0, k+1))

In [4]:
_naive_eval(1.5, spl.t, spl.c, 3), spl(1.5)

NameError: name 'spl' is not defined

In [4]:
import numpy as np
from scipy.interpolate import make_interp_spline

def prod(iterable):
    """
    Product of a sequence of numbers.

    Faster than np.prod for short lists like array shapes, and does
    not overflow if using Python integers.
    """
    product = 1
    for x in iterable:
        product *= x
    return product


def unravel_index(iflat, strides):
    """np.unravel_index replacement.
    
    NB: `strides` are element strides for indexing, not byte strides
    that numpy machinery uses!
    """
    val = iflat
    idx = []
    for i in range(len(strides)):
        #breakpoint()
        this = val // strides[i]
        rest = val % strides[i]
        idx.append(this)
        val = rest
    return tuple(idx)

    # Here's how to prepare the strides array

    # Replacement for `np.unravel_index(iflat, (k+1,)*ndim)`
    # Prepare the 'strides' (these are index strides, not byte strides!)
    strides = [-101]*ndim
    for d in range(ndim):
        strides[d] = (self.k[0] + 1)**d   # XXX: k[0] == k[1] == ... == k[ndim-1]
    strides = strides[::-1]    

In [6]:
x = np.arange(6)
y = x**3
spl = make_interp_spline(x, y, k=3)

In [6]:
spl.t

array([0., 0., 0., 0., 2., 3., 5., 5., 5., 5.])

In [ ]:
spl.c

In [ ]:
y_1 = x**3 + 2*x
spl_1 = make_interp_spline(x, y_1, k=3)
spl_1.t, spl_1.c

In [8]:
# make the outer product of 1D coefficients

c2 = np.outer(spl_1.c, spl.c)

with np.printoptions(suppress=True, linewidth=100):
    print(c2)

NameError: name 'spl_1' is not defined

In [5]:
def bspline2(xy, t, c, k):
    x, y = xy
    tx, ty = t
    nx = len(tx) - k - 1
    assert (nx >= k+1)
    ny = len(ty) - k - 1
    assert (ny >= k+1)
    return sum(c[ix, iy] * B(x, k, ix, tx) * B(y, k, iy, ty) for ix in range(nx) for iy in range(ny))

In [8]:
for x0, y0 in [(1.5, 0.5), (2.5, 1), (0.5, 1.5)]:
    print((x0**3 + 2*x0)* y0**3, ' -- ',
          bspline2((x0, y0), (spl.t, spl_1.t), c2, k=3))

NameError: name 'spl_1' is not defined

In [ ]:
(0.5**3 + 2*0.5) * 1.5**3

In [6]:
class NdBSpline0:
    def __init__(self, t, c, k=3):
        """Tensor product spline object.
        
        c[i1, i2, ..., id] * B(x1, i1) * B(x2, i2) * ... * B(xd, id)
        
        
        Parameters
        ----------
        c : ndarray, shape (n1, n2, ..., nd, ...)
            b-spline coefficients
        t : tuple of 1D ndarrays
            knot vectors in directions 1, 2, ... d
            ``len(t[i]) == n[i] + k + 1``
        k : int or length-d tuple of integers
            spline degrees.
        """
        ndim = len(t)
        assert ndim <= len(c.shape)
        
        try:
            len(k)
        except TypeError:
            # make k a tuple
            k = (k,)*ndim

        self.k = tuple(operator.index(ki) for ki in k)
        self.t = tuple(np.asarray(ti, dtype=float) for ti in t)
        self.c = c

    def __call__(self, x):
        ndim = len(self.t)
        # a single evaluation point: `x` is a 1D array_like, shape (ndim,)
        assert len(x) == ndim
        
        # get the indices in an ndim-dimensional vector
        i = ['none',]*ndim
        for d in range(ndim):
            td, xd = self.t[d], x[d]
            k = self.k[d]
            
            # find the index for x[d]
            if xd == td[k]:
                i[d] = k
            else:
                i[d] = np.searchsorted(td, xd) - 1
            assert td[i[d]] <= xd <= td[i[d]+1]
            assert i[d] >= k and i[d] < len(td) - k
        i = tuple(i)
        
        # iterate over the dimensions, form linear combinations of
        # products B(x_1) * B(x_2) * ... B(x_N) of (k+1)**N b-splines
        # which are non-zero at `i = (i_1, i_2, ..., i_N)`.
        result = 0
        iters = [range(i[d] - self.k[d], i[d] + 1) for d in range(ndim)]
        for idx in itertools.product(*iters):
            term = self.c[idx] * prod(B(x[d], self.k[d], idx[d], self.t[d]) for d in range(ndim))
            result += term
        return result

In [217]:
def _get_dtype(dtype):
    """Return np.complex128 for complex dtypes, np.float64 otherwise."""
    if np.issubdtype(dtype, np.complexfloating):
        return np.complex_
    else:
        return np.float_
    

class NdBSpline:
    """Tensor product spline object.

    c[i1, i2, ..., id] * B(x1, i1) * B(x2, i2) * ... * B(xd, id)

    Parameters
    ----------
    c : ndarray, shape (n1, n2, ..., nd, ...)
        b-spline coefficients
    t : tuple of 1D ndarrays
        knot vectors in directions 1, 2, ... d
        ``len(t[i]) == n[i] + k + 1``
    k : int or length-d tuple of integers
        spline degrees.
    """
    def __init__(self, t, c, k=3):
        ndim = len(t)
        assert ndim <= len(c.shape)
        
        try:
            len(k)
        except TypeError:
            # make k a tuple
            k = (k,)*ndim

        self.k = tuple(operator.index(ki) for ki in k)
        self.t = tuple(np.ascontiguousarray(ti, dtype=float) for ti in t)
        self.c = np.asarray(c)
        
        if len(k) != ndim:
            raise ValueError(f"len(t) = {ndim} != {len(k)} = len(k)")

        for d in range(ndim):
            td = self.t[d]
            kd = self.k[d]
            n = td.shape[0] - kd - 1
            if kd < 0:
                raise ValueError(f"Spline degree in dimension {d} cannot be negative.")
            if td.ndim != 1:
                raise ValueError(f"Knot vector in dimension {d} must be one-dimensional.")           
            if n < kd + 1:
                raise ValueError(f"Need at least {2*kd + 2} knots for degree {kd}"
                                 f" in dimension {d}.")
            if (np.diff(td) < 0).any():
                raise ValueError(f"Knots in dimension {d} must be in a non-decreasing order.")
            if len(np.unique(td[kd:n + 1])) < 2:
                raise ValueError(f"Need at least two internal knots in dimension {d}.")
            if not np.isfinite(td).all():
                raise ValueError(f"Knots in dimension {d} should not have nans or infs.")
            if self.c.ndim < ndim:
                raise ValueError(f"Coefficients must be at least {d}-dimensional.")
            if self.c.shape[d] < n:
                raise ValueError(f"Knots, coefficients and degree in dimension {d} are inconsistent:"
                                 f" got {self.c.shape[d]} coefficients for {len(td)} knots,"
                                 f" need at least {n} for k={k}.")            

        dt = _get_dtype(self.c.dtype)
        self.c = np.ascontiguousarray(self.c, dtype=dt)
                                 
        # tabulate the flat indices for iterating over the (k+1)**ndim subarray
        shape = tuple(kd + 1 for kd in self.k)
        indices = np.unravel_index( np.arange(prod(shape)), shape)
        self._indices_k1d = np.asarray(indices).T
        
    def __call__(self, xi):
        """Evaluate the tensor product b-spline at coordinates.
        
        Parameters
        ----------
        xi : array_like, shape(..., ndim)
            The coordinates to evaluate the interpolator at.
            This can be a list or tuple of ndim-dimensional points
            or an array with the shape (num_points, ndim).
            
        Returns
        -------
        values : ndarray, shape xi.shape[:-1] + self.c.shape[ndim:]
            Interpolated values at xi
        """
        ndim = len(self.t)

        # prepare xi : shape (..., m1, ..., md) -> (1, m1, ..., md)
        xi = np.asarray(xi, dtype=float)
        xi_shape = xi.shape
        xi = xi.reshape(-1, xi_shape[-1])        
        xi = np.ascontiguousarray(xi)

        if xi_shape[-1] != ndim:
            raise ValueError(f"Shapes: xi.shape={xi_shape} and ndim={ndim}")
        assert xi_shape[-1] == xi.shape[-1]
        
        # prepare the coefficients: flatten the trailing dimensions
        c1 = self.c.reshape(self.c.shape[:ndim] + (-1,))
        c1r = c1.ravel()
        assert c1r.flags.c_contiguous
        
        num_c_tr = c1.shape[-1]  # # of trailing coefficients
        out = np.empty(xi.shape[:-1] + (num_c_tr,), dtype=float)    

        # replacement for np.ravel_multi_index for indexing of `c1`
        strides_c1 = [1]*(ndim + 1)
        for d in range(ndim-1, -1, -1):
            strides_c1[d] = strides_c1[d+1] * c1.shape[d+1]
        assert strides_c1 == [_//8 for _ in c1.strides]
        assert strides_c1[-1] == 1

        # 'intervals': indices for a point in xi into the knot arrays t
        i = [-101,]*ndim
        # container for non-zero b-splines at each point in xi
        b = np.empty((ndim, max(self.k)+1), dtype=float) * np.nan

        ### Finally, iterate over the data points
        for j in range(xi.shape[0]):           
            x = xi[j]

            # get the indices in an ndim-dimensional vector
            for d in range(ndim):
                td, xd = self.t[d], x[d]
                k = self.k[d]

                # find the index for x[d]
                if xd == td[k]:
                    i[d] = k
                else:
                    i[d] = np.searchsorted(td, xd) - 1
                assert td[i[d]] <= xd <= td[i[d]+1]
                assert i[d] >= k and i[d] < len(td) - k

                # (k+1) b-splines which are non-zero at x[d] 
                b[d, :k+1] = [B(xd, k, j, td) for j in range(i[d]-k, i[d]+1)]   

            # iterate over the dimensions, form linear combinations of
            # products B(x_1) * B(x_2) * ... B(x_N) of (k+1)**N b-splines
            # which are non-zero at `i = (i_1, i_2, ..., i_N)`.
            result = np.zeros(num_c_tr, dtype=float)
            iters = [range(i[d] - self.k[d], i[d] + 1) for d in range(ndim)]
            for idx in itertools.product(*iters):
                factor = prod(b[d, idx[d] - i[d] + self.k[d]] for d in range(ndim))
                # loop over the trailing values of self.c explicitly
                for i_c in range(num_c_tr):
                    result[i_c] += c1[idx + (i_c,)] * factor
            
            
            ### The above is in principle enough for a prototype.  ###
            ### Replicate it with flat indexing below.             ###
            result_flat = np.zeros(num_c_tr, dtype=float)
            
            volume = prod([k+1 for k in self.k])  # (k+1)**ndim
            for iflat in range(volume):
                # idx_b = np.unravel_index(iflat, (k+1,)*ndim)   # equiv below
                idx_b = self._indices_k1d[iflat, :]
                assert all(idx_b == np.unravel_index(iflat, [kd+1 for kd in self.k])) # (k+1,)*ndim))
                
                # 1. Shift the subblock indices into indices into c1.ravel()
                # 2. Collect the product of non-zero b-splines at this value of the $x$ vector
                # 3. Compute the base index for iterating over the c1 array
                idx_c = [-101]*ndim
                idx_cflat_base = 0
                factor = 1.0
                for d in range(ndim):
                    factor *= b[d, idx_b[d]]
                    idx_c[d] = idx_b[d] + i[d] - self.k[d]             # XXX: remove later
                    idx_cflat_base += idx_c[d] * strides_c1[d]

                ##### double-check the above loop -- XXX: remove later
                # Product of non-zero bsplines at this value of the $x$ vector.
                factor2 = prod(b[d, idx_b[d]] for d in range(ndim))
        
                # shift the indices into c1.ravel()
                idx_c2 = list(idx_b)
                for d in range(ndim):
                    idx_c2[d] += i[d] - self.k[d] 

                idx_cflat_base2 = sum(idx_c[d] * strides_c1[d] for d in range(ndim))
                    
                assert idx_c2 == idx_c
                assert factor == factor2
                assert idx_cflat_base == idx_cflat_base2
                ####### end double-check
                    
                ### collect linear combinations of coef * factor
                for i_c in range(num_c_tr):
                    # this is equivalent to 
                    # idx_cflat = np.ravel_multi_index(tuple(idx_c) + (i_c,), c1.shape)
                    # we pre-computed the first ndim strides of `c1r` array and use the
                    # fact that the `c1r` array is C-ordered by construction
                    result_flat[i_c] += c1r[idx_cflat_base + i_c] * factor
                    out[j, i_c] = result_flat[i_c]

            # XXX: remove later
            from numpy.testing import assert_allclose
            assert_allclose(result, out[j, :], atol=1e-15)
            
            # copy the result over: this is in fact a loop over num_c_tr
            #out[j, ...] = result_flat
            
        return out.reshape(xi_shape[:-1] + self.c.shape[ndim:])

In [11]:
bspl2 = NdBSpline((spl.t, spl_1.t), c2, k=3)

for x0, y0 in [(1.5, 2.5), (2.5, 1), (0.5, 1.5)]:
    print((x0**3 + 2*x0)* y0**3, ' -- ',
          bspline2((x0, y0), (spl.t, spl_1.t), c2, k=3), '--',
         ## naive_B_2((x0, y0), (spl.t, spl_1.t), c2, k=3), )
          bspl2((x0, y0))
         )

NameError: name 'spl_1' is not defined

In [ ]:
bspl2([(1.5, 2.5), (2.5, 1), (0.5, 1.5)])

In [ ]:
spl2 = NdBSpline((spl.t, spl_1.t), c2, k=3)

spl2((1.5, 2.5))

In [ ]:
spl.t, spl_1.t

In [ ]:
k = 3
p = itertools.product(*(range(0, k+1),)*3)
for j, tpl in enumerate(p):
    #print(j, tpl, a[tpl])
    pass

# Tests

In [8]:
def bspline2(xy, t, c, k):
    """A naive 2D tensort product spline evaluation."""
    x, y = xy
    tx, ty = t
    nx = len(tx) - k - 1
    assert (nx >= k+1)
    ny = len(ty) - k - 1
    assert (ny >= k+1)
    return sum(c[ix, iy] * B(x, k, ix, tx) * B(y, k, iy, ty) for ix in range(nx) for iy in range(ny))


def naive_B_2(xy, t, c, k):
    """
    Naive B-spline evaluation, another way.
    """
    x, y = xy
    tx, ty = t
    
    if x == tx[k]:
        ix = k
    else:
        ix = np.searchsorted(tx, x) - 1
    assert tx[ix] <= x <= tx[ix + 1]
    assert ix >= k and ix < len(tx) - k

    if y == ty[k]:
        iy = k
    else:
        iy = np.searchsorted(ty, y) - 1
    assert ty[iy] <= y <= ty[iy + 1]
    assert iy >= k and iy < len(ty) - k
    
    res = sum(c[ix-jx, iy - jy] * B(x, k, ix-jx, tx) * B(y, k, iy-jy, ty)
               for jx in range(0, k+1) for jy in range(0, k+1))
    
    result = 0
    for jx in range(ix-k, ix+1):
        for jy in range(iy-k, iy+1):
            result += c[jx, jy] * B(x, k, jx, tx) * B(y, k, jy, ty)
            
    assert np.allclose(result,  res) #, result-res
    return result

In [218]:
from numpy.testing import assert_allclose
from numpy.random import default_rng


def make_2d_case():
    # make a 2D separable spline
    x = np.arange(6)
    y = x**3
    spl = make_interp_spline(x, y, k=3)
    
    y_1 = x**3 + 2*x
    spl_1 = make_interp_spline(x, y_1, k=3)

    t2 = (spl.t, spl_1.t)
    #c2 = np.outer(spl_1.c, spl.c)
    c2 = spl.c[:, None] * spl_1.c[None, :]
    
    return t2, c2, 3


def make_2d_mixed():
    # make a 2D separable spline w/ kx=3, ky=2
    x = np.arange(6)
    y = x**3
    spl = make_interp_spline(x, y, k=3)
    
    x = np.arange(5) + 1.5
    y_1 = x**2 + 2*x
    spl_1 = make_interp_spline(x, y_1, k=2)

    t2 = (spl.t, spl_1.t)
    c2 = spl.c[:, None] * spl_1.c[None, :]    
    
    return t2, c2, spl.k, spl_1.k
    

In [220]:
# test 2D

def test_2D_separable():
    print('.')
    xi = [(1.5, 2.5), (2.5, 1), (0.5, 1.5)]  
    t2, c2, k = make_2d_case()
    target = [x**3 * (y**3 + 2*y) for (x, y) in xi]
    
    # sanity check: bspline2 gives the product as constructed
    assert_allclose([bspline2(xy, t2, c2, k) for xy in xi],
                    target,
                    atol=1e-14)

    # check evaluation on a 2D array: the 1D array of 2D points
    bspl2 = NdBSpline(t2, c2, k=3)
    assert bspl2(xi).shape == (len(xi), )
    assert_allclose(bspl2(xi),
                    target, atol=1e-14)
    
    # now check on a multidim xi
    rng = default_rng(12345)
    xi = rng.uniform(size=(4, 3, 2)) * 5
    result = bspl2(xi)
    assert result.shape == (4, 3)
    
    # also check the values
    x, y = xi.reshape((-1, 2)).T
    assert_allclose(result.ravel(),
                    x**3 * (y**3 + 2*y), atol=1e-14)


def test_2D_separable_2():
    # test `c` with trailing dimensions, i.e. c.ndim > ndim
    print('.')
    ndim = 2
    xi = [(1.5, 2.5), (2.5, 1), (0.5, 1.5)]  
    target = [x**3 * (y**3 + 2*y) for (x, y) in xi]

    t2, c2, k = make_2d_case()
    c2_4 = np.dstack((c2, c2, c2, c2))   # c22.shape = (6, 6, 4)
    
    xy = (1.5, 2.5)
    bspl2_4 = NdBSpline(t2, c2_4, k=3)
    result = bspl2_4(xy)
    val_single = NdBSpline(t2, c2, k)(xy)
    assert result.shape == (4,)
    assert_allclose(result,
                    [val_single,]*4, atol=1e-14)
    
    # now try the array xi : the output.shape is (3, 4)
    # where 3 is the number of points in xi and 4 is the trailing dimension of c
    assert bspl2_4(xi).shape == np.shape(xi)[:-1] + bspl2_4.c.shape[ndim:]
    assert_allclose(bspl2_4(xi) - np.asarray(target)[:, None],
                    0, atol=5e-14)
    
    # two trailing dimensions
    c2_22 = c2_4.reshape((6, 6, 2, 2))
    bspl2_22 = NdBSpline(t2, c2_22, k=3)
    
    result = bspl2_22(xy)
    assert result.shape == (2, 2)
    assert_allclose(result,
                    [[val_single, val_single],
                     [val_single, val_single]], atol=1e-14)

    # now try the array xi : the output shape is (3, 2, 2)
    # for 3 points in xi and c trailing dimensions being (2, 2)
    assert bspl2_22(xi).shape == np.shape(xi)[:-1] + bspl2_22.c.shape[ndim:]
    assert_allclose(bspl2_22(xi) - np.asarray(target)[:, None, None],
                    0, atol=5e-14)

def test_2D_random():
    print('.')
    rng = default_rng(12345)
    k = 3
    tx = np.r_[0, 0, 0, 0, np.sort(rng.uniform(size=7)) * 3, 3, 3, 3, 3]
    ty = np.r_[0, 0, 0, 0, np.sort(rng.uniform(size=8)) * 4, 4, 4, 4, 4]
    c = rng.uniform(size=(tx.size-k-1, ty.size-k-1))
    
    spl = NdBSpline((tx, ty), c, k=k)
    
    xi = (1., 1.)
    assert_allclose(spl(xi),
                    bspline2(xi, (tx, ty), c, k), atol=1e-14)

    xi = np.c_[[1, 1.5, 2],
               [1.1, 1.6, 2.1]]
    assert_allclose(spl(xi),
                    [bspline2(xy, (tx, ty), c, k) for xy in xi], atol=1e-14)

    
def test_2D_mixed():
    print('.')
    t2, c2, kx, ky = make_2d_mixed()
    ndim = 2
    xi = [(1.4, 4.5), (2.5, 2.4), (4.5, 3.5)]
    target = [x**3 * (y**2 + 2*y) for (x, y) in xi]    

    bspl2 = NdBSpline(t2, c2, k=(kx, ky))
    assert bspl2(xi).shape == (len(xi), )
    assert_allclose(bspl2(xi),
                    target, atol=1e-14)


def test_2D_mixed_random():
    print('.')
    rng = default_rng(12345)
    kx, ky = 2, 3
    tx = np.r_[0, 0, 0, 0, np.sort(rng.uniform(size=7)) * 3, 3, 3, 3, 3]
    ty = np.r_[0, 0, 0, 0, np.sort(rng.uniform(size=8)) * 4, 4, 4, 4, 4]
    c = rng.uniform(size=(tx.size - kx - 1, ty.size - ky - 1))

    xi = np.c_[[1, 1.5, 2],
               [1.1, 1.6, 2.1]]

    bspl2 = NdBSpline((tx, ty), c, k=(kx, ky))
    bspl2_0 = NdBSpline0((tx, ty), c, k=(kx, ky))
    
    assert_allclose(bspl2(xi),
                    [bspl2_0(xp) for xp in xi], atol=1e-14)
    
#### RUN test ###
test_2D_separable()
test_2D_separable_2()
test_2D_random()
test_2D_mixed()
test_2D_mixed_random()

.
.
.
.
.


In [228]:
# test 3D

def make_3d_case():
    # make a 3D separable spline
    x = np.arange(6)
    y = x**3
    spl = make_interp_spline(x, y, k=3)
    
    y_1 = x**3 + 2*x
    spl_1 = make_interp_spline(x, y_1, k=3)

    y_2 = x**3 + 3*x + 1
    spl_2 = make_interp_spline(x, y_2, k=3)
    
    t2 = (spl.t, spl_1.t, spl_2.t)
    c2 = spl.c[:, None, None] * spl_1.c[None, :, None] * spl_2.c[None, None, :]
    
    return t2, c2, 3


def test_3D_separable():
    print('.')
    rng = default_rng(12345)
    x, y, z = rng.uniform(size=(3, 11)) * 5
    target = x**3 * (y**3 + 2*y) * (z**3 + 3*z + 1)

    t3, c3, k = make_3d_case()
    bspl3 = NdBSpline(t3, c3, k=3)

    xi = [_ for _ in zip(x, y, z)]
    result = bspl3(xi)
    assert result.shape == (11,)
    assert_allclose(result, target, atol=1e-14)
    
    
def test_3D_random():
    print('.')
    rng = default_rng(12345)
    k = 3
    tx = np.r_[0, 0, 0, 0, np.sort(rng.uniform(size=7)) * 3, 3, 3, 3, 3]
    ty = np.r_[0, 0, 0, 0, np.sort(rng.uniform(size=8)) * 4, 4, 4, 4, 4]
    tz = np.r_[0, 0, 0, 0, np.sort(rng.uniform(size=8)) * 4, 4, 4, 4, 4]
    c = rng.uniform(size=(tx.size-k-1, ty.size-k-1, tz.size-k-1))

    spl = NdBSpline((tx, ty, tz), c, k=k)
    spl_0 = NdBSpline0((tx, ty, tz), c, k=k)
    
    xi = (1., 1., 1)
    assert_allclose(spl(xi), spl_0(xi), atol=1e-14)

    xi = np.c_[[1, 1.5, 2],
               [1.1, 1.6, 2.1],
               [0.9, 1.4, 1.9]]
    assert_allclose(spl(xi), [spl_0(xp) for xp in xi], atol=1e-14)
    
    
def test_3D_random_complex():
    print('.')
    rng = default_rng(12345)
    k = 3
    tx = np.r_[0, 0, 0, 0, np.sort(rng.uniform(size=7)) * 3, 3, 3, 3, 3]
    ty = np.r_[0, 0, 0, 0, np.sort(rng.uniform(size=8)) * 4, 4, 4, 4, 4]
    tz = np.r_[0, 0, 0, 0, np.sort(rng.uniform(size=8)) * 4, 4, 4, 4, 4]
    c = rng.uniform(size=(tx.size-k-1, ty.size-k-1, tz.size-k-1))
    c += c*1j
    
    spl = NdBSpline((tx, ty, tz), c, k=k)
    spl_re = NdBSpline0((tx, ty, tz), c.real, k=k)
    spl_im = NdBSpline0((tx, ty, tz), c.imag, k=k)
    
    xi = np.c_[[1, 1.5, 2],
               [1.1, 1.6, 2.1],
               [0.9, 1.4, 1.9]]
    assert_allclose(spl(xi),
                    spl_re(xi) + 1j*spl_re[spl_0(xp) for xp in xi], atol=1e-14)

    
#### RUN test ###
test_3D_separable()
test_3D_random()

.
.


### Scratch area

In [261]:
ndim = 3
k = 3


arr = np.arange((k+1)**ndim, dtype=float).reshape((k+1,)*ndim)

strides = [-101]*ndim
for d in range(ndim):
    strides[d] = (k+1)**d
strides = strides[::-1]
print(strides)

for iflat in range((k+1)**ndim):
    assert np.unravel_index(iflat, (k+1,)*ndim) == unravel_index(iflat, strides)


[16, 4, 1]


In [293]:
arr = np.zeros((12, 11, 5, 4))
flat_idx = np.ravel_multi_index((4, 1, 2, 0), arr.shape)
print( flat_idx )
print( np.unravel_index(flat_idx, arr.shape))

908
(4, 1, 2, 0)


In [296]:
[_//8 for _ in arr.strides]

[220, 20, 4, 1]

In [297]:
ndim = arr.ndim - 1
strides_c1 = [1]*(ndim + 1)
print(strides_c1)
for d in range(ndim-1, -1, -1):
    strides_c1[d] = strides_c1[d+1] * arr.shape[d+1]
strides_c1

[1, 1, 1, 1]
2
1
0


[220, 20, 4, 1]

In [295]:
arr.shape

(12, 11, 5, 4)

In [32]:
k, ndim = 3, 3
np.unravel_index( np.arange(k+1)**ndim, (k+1,)*ndim)

(array([0, 0, 0, 1]), array([0, 0, 2, 2]), array([0, 1, 0, 3]))

In [39]:
indices = np.asarray( np.unravel_index( np.arange((k+1)**ndim), (k+1,)*ndim) )

In [40]:
indices

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
       [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 0, 0, 0, 0, 1, 1,
        1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2,
        3, 3, 3, 3, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
       [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1,
        2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3,
        0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]])

In [42]:
indices.shape

(3, 64)

In [44]:
indices[:, -1]

array([3, 3, 3])

In [207]:
np.ascontiguousarray([1, 2, 3], dtype=float)

array([1., 2., 3.])

In [227]:
a, b = 3, 4
s = (f"a = {a},"
     f" b = {b}")
s

'a = 3, b = 4'